In [1]:
import numpy as np
import time

## Função GE:
*inputs*: matriz inicial e lista de blocos aplicada
<br> *output*: group efficiency do agrupamento

In [2]:
def ge(mi,list_of_blocks):
    nee = np.sum(mi)
    '''n1 =0
    mrxnr= 0
    for block in list_of_blocks:
        nee -= np.sum(block)
        n1 += np.sum(block)
        mrxnr += block.shape[0]*block.shape[1]

    nee = np.sum(mi)'''
    n1 =0
    mrxnr= 0
    for block in list_of_blocks:
        n1 += np.count_nonzero(block)
        mrxnr += block.size
    nee -=n1
    pee = nee/np.sum(mi)
    #ex = pee
    es = n1/mrxnr
    ge = ((1-pee)+(es))/2

    return ge

## Funções auxiliares:
*index_to_array*: transforma um índice ("mapa da matriz") de uma matriz em numpy array 
<br> *list_to_alt*: transforma uma lista de indíces em lista de alternativas
<br> *swap_alt*: "troca" índices em uma alternativa de blocos. 

In [3]:
def index_to_array(matrix,index):
    return matrix[index[0]:index[1],index[2]:index[3]]
def list_to_alt(matrix,list_index):
    output = []
    [output.append(index_to_array(matrix,ind)) for ind in list_index]
    return output
def swap_alt(list_,swap,index):
    c = list_[:index]
    c.extend(swap)
    c.extend(list_[index+1:])
    return c

## Função breaking_matrix
*inputs*: matriz a ser "quebrada" e uma lista contendo esta matriz.
<br>*outputs*: melhor GE e melhor alternativa encontrados.


**Esta função funciona recursivamente, então reaplica-se internamente para quebrar em todas as opções possíveis de tamanho.** 

In [4]:
def matrix_break1(temp_list,matrix,max_ge=0,best_alt=list()):
    for p in range(len(temp_list)):
        m_ = temp_list[p]
        rows,cols = m_.shape[0],m_.shape[1]
        for i in range(rows-1):
            for j in range(cols-1):
                temp_list_copy = temp_list.copy()
                alt = [[0,i+1,0,j+1],[i+1,rows,j+1,cols]]
                aa =swap_alt(temp_list_copy,list_to_alt(m_,alt),p)
                #ge_alt = 10
                ge_alt = ge(matrix,aa)
                if ge_alt>max_ge:
                    max_ge,best_alt = ge_alt,aa
                max_ge2,aa2 = matrix_break1(aa,matrix)
                if max_ge2 > max_ge:
                    max_ge,best_alt = max_ge2,aa2

    return max_ge,best_alt  

In [5]:
def matrix_break2(temp_list,matrix,max_ge=0,best_alt=list(),last_ge = 0):
    for p in range(len(temp_list)):
        m_ = temp_list[p]
        rows,cols = m_.shape[0],m_.shape[1]
        for i in range(rows-1):
            for j in range(cols-1):
                temp_list_copy = temp_list.copy()
                alt = [[0,i+1,0,j+1],[i+1,rows,j+1,cols]]
                aa =swap_alt(temp_list_copy,list_to_alt(m_,alt),p)
                #ge_alt = 10
                ge_alt = ge(matrix,aa)
                if ge_alt>max_ge:
                    max_ge,best_alt = ge_alt,aa
                if ge_alt > last_ge:
                    max_ge2,aa2 = matrix_break2(aa,matrix,last_ge = ge_alt)
                    if max_ge2 > max_ge:
                        max_ge,best_alt = max_ge2,aa2

    return max_ge,best_alt  

### TESTE
Aqui crio uma matriz de tamanho 7x7 aleatória e aplico o programa desenvolvido.

In [6]:
np.random.seed(1)
i = 7
m_test = np.random.randint(2,size=(i,i))

print("Matriz teste:")
print(m_test)

Matriz teste:
[[1 1 0 0 1 1 1]
 [1 1 0 0 1 0 1]
 [1 0 0 1 0 0 0]
 [1 0 0 1 0 0 0]
 [1 0 0 0 1 1 1]
 [1 1 0 0 0 1 1]
 [1 1 1 1 0 1 1]]


In [8]:
ge_m,alt_m = matrix_break2([m_test],m_test)
print('A melhor alternativa de blocos é:')
print(alt_m)
print('\n')
print('E seu GE é de :'+str(ge_m))

A melhor alternativa de blocos é:
[array([[1],
       [1],
       [1],
       [1],
       [1],
       [1]]), array([[1, 1, 1, 0, 1, 1]])]


E seu GE é de :0.662037037037037


## TESTE DE TEMPO

In [9]:
#max_size é o número máximo de linhas e colunas da matriz testada
#se max_size = n, o programa irá rodar matrizes 1x1,2x2,3x3...nxn
#seu output será o tempo de operação e número da matrix que está sendo aplicada
#exemplo: 0.41 5 - tempo de 0.41 segundos para aplicar achar a melhor alternativa para uma matriz 5x5
max_size = 5
for i in range(50):
    for i in range(8):
        m_test = np.random.randint(2,size=(i,i))
        #np.random.seed(i)
        ge1 = matrix_break1([m_test],m_test)[0]
        ge2 = matrix_break2([m_test],m_test)[0]
        if (ge1 == ge2) == False:
            print(False)

C:\Users\vicenteneto\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in long_scalars


In [35]:
i = 15
m_test = np.random.randint(2,size=(i,i))
start_time = time.time()
matrix_break2([m_test],m_test)
print(time.time()-start_time,i)

0.4971492290496826 15


In [49]:
a = np.matrix('1 0 0 0 0 0 0 0 0; 0 1 0 0 0 0 0 0 0; 0 0 1 0 0 0 0 0 0; 0 0 0 1 0 0 0 0 0; 0 0 0 0 1 1 0 0 0; 0 0 0 0 0 1 1 0 0; 0 0 0 0 0 1 1 0 0; 0 0 0 0 0 0 0 1 1; 0 0 0 0 0 0 0 1 1')
print(a)

[[1 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0]
 [0 0 0 0 1 1 0 0 0]
 [0 0 0 0 0 1 1 0 0]
 [0 0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 0 1 1]]


In [50]:
matrix_break2([a],a)

(0.9642857142857143,
 [matrix([[1]]),
  matrix([[1]]),
  matrix([[1]]),
  matrix([[1]]),
  matrix([[1]]),
  matrix([[1, 1],
          [1, 1]]),
  matrix([[1, 1],
          [1, 1]])])